In [66]:
import pandas as pd

In [67]:
def calc_change(data):

    # function that calculates percentage change 
    
    percentage_changes = (data.pct_change() * 100).dropna()
    return round(percentage_changes, 2)

In [68]:
def clean_gdp():
  
    nom_gdp = pd.read_csv(
            'https://www.econdb.com/api/series/GDPES/?format=csv&token=b16dd00e80fab104e4e9109c2c7e681e4f0b59fa',
            index_col='Date', parse_dates=['Date'])


    real_gdp = pd.read_csv(
        'https://www.econdb.com/api/series/RGDPES/?format=csv&token=c9a2a2339603ec44ecb1ee8f6cea88f510a3fd2d',
        index_col='Date', parse_dates=['Date'])
    

    # Merge nominal and real GDP, group by year, calculate YTY change 
    # (transform from quarterly data to yearly)
     
    gdp_df = pd.merge(nom_gdp, real_gdp, on='Date', how='inner')

    gdp_df["Year"] = gdp_df.index.year

    yearly_gdp = pd.DataFrame(gdp_df.groupby(gdp_df['Year'])['GDPES'].agg('last'))

    yearly_gdp["GDP change"] = calc_change(yearly_gdp["GDPES"])

    yearly_gdp["RGDPES"]= gdp_df.groupby(gdp_df['Year'])['RGDPES'].agg('last')

    yearly_gdp["RGDP change"] = calc_change(yearly_gdp["RGDPES"])

    yearly_gdp.reset_index(inplace = True)

    yearly_gdp.to_csv("yearly_gdp.csv", index = False)

    # returns df with EOY Nominal GDP, Real GDP and % change for both

    return yearly_gdp

In [69]:
clean_gdp()

,Year,GDPES,GDP change,RGDPES,RGDP change
0,1995,117799,NaN,180677,NaN
1,1996,124949,6.07,185641,2.75
2,1997,134821,7.90,194117,4.57
3,1998,143999,6.81,202094,4.11
4,1999,153809,6.81,211652,4.73
5,2000,166992,8.57,222349,5.05
6,2001,180040,7.81,230061,3.47
7,2002,191698,6.48,236140,2.64
8,2003,205584,7.24,243245,3.01
9,2004,221355,7.67,251133,3.24


In [70]:
def clean_ibex():

    ibex = pd.read_csv("/Users/victorbescos/Data_analysis/projects/project_2/src/Datos históricos IBEX 35 (1).csv", decimal=',')

    ibex.drop(["Apertura", "Máximo", "Mínimo", "Vol.", "% var."], axis = 1, inplace = True)

    ibex['Close'] = ibex['Último'].str.replace('.', '').str.replace(',', '.').astype(float)

    ibex['Year'] = pd.to_datetime(ibex['Fecha'])

    # New df with yearly return data

    yearly_close = ibex.groupby(ibex['Year'].dt.year)['Close'].agg('last').reset_index()

    yearly_close["IBEX change"] = calc_change(yearly_close["Close"])

    yearly_close.to_csv('yearly_close.csv', index=False)

    return yearly_close


In [71]:
clean_ibex()

,Year,Close,IBEX change
0,1995,3024.54,NaN
1,1996,3734.49,23.47
2,1997,5327.42,42.65
3,1998,7958.99,49.40
4,1999,9878.80,24.12
5,2000,10835.10,9.68
6,2001,10116.00,-6.64
7,2002,8050.40,-20.42
8,2003,5947.70,-26.12
9,2004,7929.90,33.33


In [78]:
def clean_salary(): 

    salary = pd.read_csv("/Users/victorbescos/Data_analysis/projects/project_2/src/salaries.csv")
    
    salary[['Year', 'Salary']] = salary["Year;Av. salary "].str.split(';', expand=True)

    salary = salary.drop("Year;Av. salary ", axis = 1)

    salary["Salary"] = pd.to_numeric(salary['Salary'])
    
    salary["Year"] = pd.to_numeric(salary["Year"]).sort_values()

    salary.sort_index(ascending=True, inplace=True)

    # Import CPI data

    ipc = pd.read_csv(
        'https://www.econdb.com/api/series/CPIES/?format=csv&token=056e389202cbd11e217d8de0ca8c388dcba19d50',
        index_col='Date', parse_dates=['Date'])
    
    ipc["CPIES"] = pd.to_numeric(ipc["CPIES"])

    ipc["Year"] = ipc.index.year

    # Establish CPI baseline year, create yearly CPI dataframe, transform
    # into CPI in relation to baseline year
    
    baseline_year = 2002

    yearly_cpi = ipc.groupby(ipc['Year'])['CPIES'].agg('last').reset_index()

    baseline_cpi = yearly_cpi.loc[yearly_cpi['Year'] == baseline_year, 'CPIES'].values[0]

    # add column with CPI to baseline year
    
    yearly_cpi['CPI to Baseline'] = round((yearly_cpi['CPIES'] / baseline_cpi) * 100, 2)

    # new df with nominal salary and yearly cpi

    adjusted_wages = pd.merge(salary, yearly_cpi, on='Year', how='inner')

    # add column to new df: real salary calculation dividing nominal salary by CPI to baseline column

    adjusted_wages['Real Salary'] = round((adjusted_wages['Salary'] / adjusted_wages['CPI to Baseline']) * 100, 3)

    adjusted_wages = adjusted_wages.sort_values(by='Salary', ascending=True)

    adjusted_wages = adjusted_wages.reset_index(drop=True)

    adjusted_wages.to_csv("yearly_wages.csv", index = False)

    return adjusted_wages


In [79]:
clean_salary()

,Year,Salary,CPIES,CPI to Baseline,Real Salary
0,2002,18.601,72.41,100.00,18.601
1,2003,19.385,74.29,102.60,18.894
2,2004,20.045,76.69,105.91,18.926
3,2005,20.616,79.56,109.87,18.764
4,2006,21.168,81.68,112.80,18.766
5,2007,21.989,85.12,117.55,18.706
6,2008,23.252,86.35,119.25,19.499
7,2009,24.164,87.03,120.19,20.105
8,2010,24.786,89.63,123.78,20.024
9,2011,25.515,91.76,126.72,20.135
